# Capital Structure Arbitrage

Capital structure arbitrage is the term used to describe the fashion for arbitraging equity claims against fixed income and convertible claims. At its most sophisticated, practitioners build elaborate models of the capital structure of a company to determine the relative values of the various claims—in particular, stock, bonds, and convertible bonds. At its simplest, the trader looks to see if equity puts are cheaper than credit derivatives and if so buys the one and sells the other

Denoting the value of a risk-free put, call and bond by $P_0$, $C_0$, and $B_0$ and the value of risky claims on the issuer (I) of the stock by $P_I$, $C_I$, and $B_I$, we obtain

the risk-free put is worth more than the risky put. The excess value is equal to the difference in risky and risk-free bond prices (times the strike price). With maturity-independent rates and credit spreads for clarity and setting t = 0, we obtain



$B_0-B_I=e^{-r T}\left(1-e^{-\lambda T}\right)$


In [1]:
import sympy as smp

In [2]:
B_0,B_1,r,T,lambda_ = smp.symbols('B_0 B_1 r T lambda_', real=True) 
expr = B_0 - B_1 
expr

B_0 - B_1

In [3]:
 #for writing equation using smp.Ex()
smp.Eq(expr,smp.exp(-r*T)*(1-smp.exp(-lambda_*T)))


Eq(B_0 - B_1, (1 - exp(-T*lambda_))*exp(-T*r))

the extra value is the strike price times the (pseudo-) probability that default occurs. This payoff is also more or less exactly the payoff of a default put in the credit derivatives market.\
the trader buys an equity option on the exchange at a ‘‘very high’’ implied volatility and sells a default put on the same stock in the credit derivatives market locking in a risk-free return.\
 buy one at-the-money put and sell two puts struck at half the current stock price

\begin{aligned}
&\begin{aligned}
\sigma_{l o c}^2(K, T, S) & =\sigma^2-\lambda \frac{K \frac{\partial C}{\partial K}}{\frac{1}{2} K^2 \frac{\partial^2 C}{\partial K^2}} \\
& =\sigma^2+2 \lambda \sigma \sqrt{T} \frac{N\left(d_2\right)}{N^{\prime}\left(d_2\right)}
\end{aligned}\\
&d_2=\frac{\log S / K+\lambda T}{\sigma \sqrt{T}}-\frac{\sigma \sqrt{T}}{2}
\end{aligned}

Then, for very low strikes,

$\sigma_{l o c}^2(K, T, S) \approx \sigma^2+2 \lambda \sigma \sqrt{T} \sqrt{2 \pi} e^{+d_2^2 / 2}$

It follows that implied volatility in the jump-to-ruin model increases very fast as the strike decreases from at-the-money and tends to the constant σ for high strikes

In [4]:
sigma_loc_squared = smp.symbols('sigma_loc_squared', cls=smp.Function) 
sigma,K,d_2, S, f_d_2, mu_stock, sigma_stock = smp.symbols('sigma K d_2 S f_d_2 mu_stock sigma_stock', real=True) 

In [6]:
from sympy.stats import P, E, variance, Die, Normal
from sympy import simplify
f_d_2 = Normal('d_2',mu_stock, sigma_stock) 
F_d_2 = simplify(P(f_d_2 <= T))
d_2 = (smp.log(S/K) + lambda_ * T) / (sigma * smp.sqrt(T)) - ((sigma * smp.sqrt(T)) / 2)
sigma_loc_squared = sigma**2 + 2 * lambda_ * sigma * smp.sqrt(T) * (F_d_2 / f_d_2)
sigma_loc_squared

2*sqrt(T)*lambda_*sigma*Piecewise((erf(sqrt(2)*(T - mu_stock)/(2*sigma_stock))/2 + 1/2, (Abs(arg(sigma_stock)) < pi/4) | ((Abs(arg(sigma_stock)) <= pi/4) & (Abs(2*arg(T) - 4*arg(sigma_stock) + 2*arg((T - mu_stock)/T) + 2*pi) < pi))), (sqrt(2)*Integral(exp(-(_z + T - mu_stock)**2/(2*sigma_stock**2)), (_z, -oo, 0))/(2*sqrt(pi)*sigma_stock), True))/d_2 + sigma**2

fair price of a zero coupon bond of GT (assuming zero rates) should be given by

$P_t=e^{-\lambda t} R+\left(1-e^{-\lambda t}\right)$

In [7]:
P_t = smp.symbols('P_t', cls=smp.Function) 
t,R = smp.symbols('t R', real=True) 
P_t = smp.exp(-lambda_*t)*R+(1-smp.exp(-lambda_*t))

where R is the recovery rate\
most of the volatility skew for stocks with high credit spreads can be ascribed to default risk.

## Model setup

The level of V at which the company defaults is given by L D where D is
today’s value of its debt (per share) and L is the recovery rate. As discussed above, it is further assumed that the recovery rate L is a lognormally distributed random variable with mean $\bar{L}$ and standard deviation λ so that

$L D=\bar{L} D e^{\lambda Z-\lambda^2 / 2}$\
where Z ∼ N(0, 1). The random variable Z is assumed to be independent of $W_t$.

In [9]:
L,D,barL,lambda_dev = smp.symbols('L D barL lambda_dev', real=True) 
Z = Normal('Z',0, 1) 
exprr = L*D
smp.Eq(exprr,barL*D*smp.exp((lambda_dev*Z-lambda_dev**2)/2))


Eq(D*L, D*barL*exp(-lambda_dev**2/2 + lambda_dev*Z/2))

## Survival Probability

$X_t:=\sigma W_t-\lambda Z-\frac{\sigma^2 t}{2}-\frac{\lambda^2}{2}$\
\
Then $X_t$ is normally distributed with\
\
$\begin{aligned} \mathbb{E}\left[X_t\right] & =-\frac{\sigma^2}{2}\left(t+\frac{\lambda^2}{\sigma^2}\right) \\ \operatorname{Var}\left[X_t\right] & =\sigma^2\left(t+\frac{\lambda^2}{\sigma^2}\right)\end{aligned}$\
Default occurs when\
\
$V=V_0 e^{\sigma W_t-\sigma^2 t / 2}=L D=\bar{L} D e^{\lambda Z-\lambda^2 / 2}$\
\
or equivalently when\
\
$X_t=\log \left(\frac{\bar{L} D}{V_0}\right)-\lambda^2$

In [19]:
X_t,sigma,W_t,E_X_t,Var_X_t,V,V_0 = smp.symbols('X_t sigma W_t E_X_t Var_X_t V V_0', real=True) 
X_t = sigma*W_t-lambda_dev*Z-((sigma**2*t)/2) - (lambda_dev**2/2)
E_X_t = -(sigma**2/2)*(t+(lambda_dev**2)/2)
Var_X_t = sigma**2*(t+(lambda_dev**2)/2)
V = V_0 * smp.exp(sigma*W_t-sigma**2*t/2)

#Default case
if V == exprr or X_t == smp.log((barL*D)/V_0)-lambda_dev**2:
    print("Default")
V



V_0*exp(W_t*sigma - sigma**2*t/2)

Since $\hat{X}$ is a Brownian motion with drift, the probability of survival (or the probability of not hitting the default barrier) is given by the Black-Scholes- like formula


$P_t=N\left(-\frac{A_t}{2}+\frac{\log d}{A_t}\right)-d N\left(-\frac{A_t}{2}-\frac{\log d}{A_t}\right)$\
\
with\
\
$d=\frac{V_0 e^{\lambda^2}}{\bar{L} D} ; A_t^2=\sigma^2 t+\lambda^2$\
\
Since $P_t$ is the probability of survival up to time t, it may be estimated directly from the prices of risky instruments such as bonds and credit default swaps (CDS).


In [37]:
A_t_squared,d,A_t = smp.symbols('A_t_squared d A_t', real=True) 
P_t = smp.symbols('P_t', cls=smp.Function)
exprs1 = Normal('expr1',0,1 )
exprs2 = Normal('expr2',0,1 )
expr1 = (-A_t/2) + (smp.log(d)/A_t) 
expr2 = (-A_t/2) - (smp.log(d)/A_t) 
d = V_0 * smp.exp(lambda_dev**2) / (L*D) 
A_t_squared = sigma**2 * t + lambda_dev**2
P_t = exprs1 - d * exprs2
P_t

expr1 - V_0*exp(lambda_dev**2)*expr2/(D*L)

## Equity volatility

The stock price S is approximately related (neglecting the time value of the option) to the firm value V via
$V \approx L D+S$ then\
\
$\sigma \sim \frac{\delta V}{V} \approx \frac{\delta S}{S+L D}=\frac{\delta S}{S} \frac{S}{S+L D} \sim \sigma_S \frac{S}{S+L D}$\
\
where $σ_S$ is the stock volatility. We see that as the stock price rises, keeping σ fixed, the volatility $σ_S$ of the stock declines. Conversely, as the stock price S approaches zero, the stock volatility increases as 1/S.

## Model calibration

We end up with the following model in terms of market observables\
\
$P_t=N\left(-\frac{A_t}{2}+\frac{\log d}{A_t}\right)-d N\left(-\frac{A_t}{2}-\frac{\log d}{A_t}\right)$\
\
$d=\frac{S_0+\bar{L} D}{\bar{L} D} e^{\lambda^2} ; A_t^2=\left(\sigma_S^* \frac{S^*}{S^*+\bar{L} D}\right)^2 t+\lambda^2$\
\
where $S^*$ is some reference stock price and $σ_{S}^*$ the stock volatility at that price.

In [40]:
S_0,S_t = smp.symbols('S_0 S_t', real=True)
expr1 = (-A_t/2) + (smp.log(d)/A_t) 
expr2 = (-A_t/2) - (smp.log(d)/A_t) 
d = ((S_0 + barL*D ) / (L*D)) *smp.exp(lambda_dev**2)
A_t_squared = (sigma_stock**2 * (S_t/(S_t + L*D))**2)*t + lambda_dev**2


Getting $\bar{L}$, λ and D from company and industry data rather than from the term structure of credit spreads would theoretically enable us to identify rich and cheap claims.

In [ ]:
# To evaluate the equation use the expression: 
A_t_squared.subs((sigma_stock,sigma_stock),(S_t,S_t),(L,L),(D,D),(t,t),(lambda_dev,lambda_dev))

> all the text and formulas are taken from "The Volatility Surface: A Practitioner's Guide" by Jim Gatheral